# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [466]:
# import numpy and pandas
import pandas as pd
import numpy as np
import scipy.stats
from statsmodels.stats.weightstats import ztest as ztest #Z (z o Pvalue)
from statsmodels.stats.weightstats import zconfint as zconfint #IC
from scipy.stats import ttest_1samp #T (T o Pvalue)
from scipy.stats import t # T interval
from statsmodels.stats.proportion import proportions_ztest # proportions_ztest

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Please, load the data using Ironhack's database (db: employees, table: employees_advanced).

In [41]:
# Your code here:
df1 = pd.read_csv("datasets/Current_Employee_Names__Salaries__and_Position_Titles.csv")

Examine the `salaries` dataset using the `head` function below.

In [42]:
# Your code here:
df1.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [45]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


In [43]:
df1_salary = df1[df1["Salary or Hourly"] == "Salary"]
del df1_salary["Typical Hours"]
del df1_salary["Hourly Rate"]
df1_salary = df1_salary.reset_index(drop=True)
df1_salary.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Annual Salary
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,101442.0
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,94122.0
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,101592.0
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,110064.0
4,"ABBASI, CHRISTOPHER",STAFF ASST TO THE ALDERMAN,CITY COUNCIL,F,Salary,50436.0


In [416]:
# Replace values:
df1_hourly = df1[df1["Salary or Hourly"] == "Hourly"]
del df1_hourly["Annual Salary"]

df1_hourly["Hourly Salary"] = df1_hourly.apply(lambda row: (row['Typical Hours']*row['Hourly Rate']),axis=1)
df1_hourly = df1_hourly.reset_index(drop=True)
df1_hourly.head()

C:\Users\AlvaroSaez\miniconda3\envs\ironhack\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Hourly Rate,Hourly Salary
0,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,19.86,397.2
1,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,46.10,1844.0
2,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,35.60,1424.0
3,"ABBOTT, BETTY L",FOSTER GRANDPARENT,FAMILY & SUPPORT,P,Hourly,20.0,2.65,53.0
4,"ABDULLAH, LAKENYA N",CROSSING GUARD,OEMC,P,Hourly,20.0,17.68,353.6


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [51]:
print("nº salaried employees: "+str(len(df1_salary)))
print("nº hourly employees: "+str(len(df1_hourly)))

nº salaried employees: 25161
nº hourly employees: 8022


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [60]:
# Your code here:
employees_by_dep_serie = df1["Department"].value_counts()
employees_by_dep_serie

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

## Exercise 1

In [164]:
# bilateral Hypothesis Tests. We will use de hourly workers df
#population
μ = 30 # hourly wage of population,main Hypothesis  column "Typical Hours"
H0 = μ

#sample
h1 = df1_hourly["Hourly Rate"].mean() #32,79
n = len(df1_hourly) #8022
s = df1_hourly["Hourly Rate"].std() #12.11 STD σ
α = 0.05
n_sqrt = np.sqrt(n)

#### With Z (bilateral)

######          a ) manual formula

In [165]:
# manual Z
z_distribution = scipy.stats.norm.ppf(1-α/2)
z_distribution_range= (-z_distribution,z_distribution)
z_manual = (h1-H0)/(s/n_sqrt)

In [168]:
print("Z is: " + str(z_manual))
print("Z distribution is: " + str(z_distribution_range))
"""Z is not in the aceptance region, so we reject the main Hypothesis"""

Z is: 20.619805785501907
Z distribution is: (-1.959963984540054, 1.959963984540054)


'Z is not in the aceptance region, so we reject the main Hypothesis'

######          b ) python library formula

In [188]:
# formula Z
z_formula, p_value_formula = ztest(x1=df1_hourly["Hourly Rate"], value = H0)
print("Z using a 'ztest formula' is: " + str(z_formula))
print("Z distribution is: : " + str(ICZ))
"""Z is not in the aceptance region, so we reject the main Hypothesis"""

Z using a 'ztest formula' is: 20.6198057854942
Z distribution is: : (-1.959963984540054, 1.959963984540054)


'Z is not in the aceptance region, so we reject the main Hypothesis'

#### PValue

######          a ) PVALUE ztest

In [189]:
#PVALUE fórmula con alfa
z_formula, p_value_formula = ztest(x1=df1_hourly["Hourly Rate"], value = H0)
p_value_formula

1.8228873859286195e-94

######          b) PVALUE scipy.stats

In [190]:
#PVALUE fórmula con Z
scipy.stats.norm.sf(z_formula)*2

1.8228873859286195e-94

In [200]:
"""Pvalue is lower than Alfa, so we reject the main Hypothesis"""

'Pvalue is lower than Alfa, so we reject the main Hypothesis'

#### Just to learn with T (bilateral)

######          a) T with ttest_1samp

In [355]:
t_formula, t_pvalue = ttest_1samp(df1_hourly["Hourly Rate"], H0)

In [356]:
t_formula

20.6198057854942

In [349]:
#BILATERAL T CRITICAL VALUE aceptance region
bi_t_izq = scipy.stats.t.ppf((0.05)/2,len(df1_hourly)-1) #IZQ
bi_t_dch = scipy.stats.t.ppf((1-0.05/2),len(df1_hourly)-1) #DERECHA
(bi_t_izq,bi_t_dch)

(-1.960259785956168, 1.9602597859561675)

In [357]:
"""rechazamos"""

'rechazamos'

######          b.1) PValue with ttest_1samp

In [358]:
t_pvalue

4.3230240486229894e-92

In [359]:
"""rechazamos"""

'rechazamos'

######          b.2) PValue with scipy.stats

In [381]:
#scipy.stats.t.sf(abs(-t_pvalue), len(df1_hourly)-1)*2
scipy.stats.t.sf(abs(t_pvalue), len(df1_hourly)-1)*2

1.0

In [382]:
"""tiene pinta de estar mal"""

'tiene pinta de estar mal'

## Exercise 2

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [126]:
# DATAFRAME OF OUR SAMPLE:
df_salaries_police_dept = df1_salary[df1_salary["Department"] == "POLICE"].reset_index(drop=True)
df_salaries_police_dept.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Annual Salary
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,101442.0
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,94122.0
2,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,93354.0
3,"ABDALLAH, ZAID",POLICE OFFICER,POLICE,F,Salary,84054.0
4,"ABDELHADI, ABDALMAHD",POLICE OFFICER,POLICE,F,Salary,87006.0


In [140]:
#UNILATERAL TEST
#population
μ_police = 86000 
H0_police = μ_police

#sample
#hypothesis h1 > H0
h1_police = df_salaries_police_dept["Annual Salary"].mean() #86486.41
n_police = len(df_salaries_police_dept) #13404
s_police = df_salaries_police_dept["Annual Salary"].std() #18272.23 STD σ
α_police = 0.05
IC_police = 1-α
n_sqrt_police = np.sqrt(n)

#### With Z (Unilateral)

######          a ) python library formula

In [144]:
z_distribution_police =scipy.stats.norm.ppf(1-α_police)
z_police, p_value_police = ztest(x1=df_salaries_police_dept["Annual Salary"], value = H0_police, alternative="larger")

In [145]:
z_distribution_police

1.6448536269514722

In [146]:
z_police

3.081997005712994

In [154]:
"""REGIÓN DE ACEPTACIÓN DE Z para este test unilateral es rechazar lo que está a la derecha de 1.64. En nuestro caso Z = 3.08 está a la DERECHA por lo que RECHAZAMOS nuestra hipótesis"""

'REGIÓN DE ACEPTACIÓN DE Z para este test unilateral es rechazar lo que está a la derecha de 1.64. En nuestro caso Z = 3.08 está a la DERECHA por lo que RECHAZAMOS nuestra hipótesis'

######          b ) Pvalue

In [147]:
p_value_police

0.0010280845287547855

In [467]:
"""P value es 0.001028. Si es menor al nivel de significancia (alfa = 0,05) RECHAZAMOS la hipótesis. Como es menor RECHAZAMOS LA HIPÓTESIS"""

'P value es 0.001028. Si es menor al nivel de significancia (alfa = 0,05) RECHAZAMOS la hipótesis. Como es menor RECHAZAMOS LA HIPÓTESIS'

#### just to learn with T (unilateral)

######          a ) python library formula

In [367]:
t_formula_police, t_pvalue_police = ttest_1samp(df_salaries_police_dept["Annual Salary"], H0_police, alternative="greater")

In [368]:
t_formula_police

3.081997005712994

In [378]:
#UNILATERAL T CRITICAL VALUE aceptance region
uni_t_menor = scipy.stats.t.ppf((0.05),len(df_salaries_police_dept)-1) # H1 < H0
uni_t_mayor = scipy.stats.t.ppf((1-0.05),len(df_salaries_police_dept)-1) # H1 > H0
(uni_t_menor,uni_t_mayor)

(-1.6449673235204962, 1.6449673235204956)

In [372]:
"""rechazamos"""

'rechazamos'

######          b.1 ) Pvalue with ttest_1samp

In [370]:
t_pvalue_police

0.0010301701775482569

In [373]:
"""rechazamos"""

'rechazamos'

######          b.2 ) Pvalue with scipy.stats

In [379]:
#scipy.stats.t.sf(abs(-T), len(df_salaries_police_dept)-1) # H1 < H0
scipy.stats.t.sf(abs(t_pvalue_police), len(df_salaries_police_dept)-1)  # H1 > H0

0.49958902929858373

In [380]:
"""NO SE SI ESTÁ BIEN"""

'NO SE SI ESTÁ BIEN'

## Exercise 3

Using the `crosstab` function, find the department that has the most hourly workers. 

###### Pivot Table "most hours amount"

In [269]:
# DEPARTMENT WITH MORE HOURS
df_most_hourly_hours = pd.pivot_table(df1_hourly, index=["Department"],values=["Typical Hours"], aggfunc="sum").sort_values(by="Typical Hours", ascending=False)
df_most_hourly_hours.iloc[0:1]

,Typical Hours
Department,
STREETS & SAN,69150.0


###### Pivot Table "most hourly workers"

In [270]:
# DEPARTMENT WITH MORE WORKERS
df_most_hourly_hours = pd.pivot_table(df1_hourly, index=["Department"],values=["Typical Hours"], aggfunc="count").sort_values(by="Typical Hours", ascending=False)
df_most_hourly_hours.iloc[0:1]

,Typical Hours
Department,
STREETS & SAN,1862


###### crosstab "most hourly workers"

In [296]:
# DEPARTMENT WITH MORE WORKERS USING CROSSTAB
dfcross_most_hourly_workers = pd.crosstab(df1_hourly["Department"],df1_hourly["Typical Hours"],margins=True, margins_name="Total").sort_values(by="Total", ascending=False)
dfcross_most_hourly_workers.iloc[1:2]["Total"]

Department
STREETS & SAN    1862
Name: Total, dtype: int64

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [420]:
df_dept_most_hourly = df1_hourly[df1_hourly["Department"] == "STREETS & SAN"].reset_index(drop=True)
df_dept_most_hourly.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Hourly Rate,Hourly Salary
0,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,35.60,1424.0
1,"ABDUL-SHAKUR, TAHIR",GENERAL LABORER - DSS,STREETS & SAN,F,Hourly,40.0,21.43,857.2
2,"ABERCROMBIE, TIMOTHY",MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,35.60,1424.0
3,"ABRAMS, DANIELLE T",SANITATION LABORER,STREETS & SAN,F,Hourly,40.0,36.21,1448.4
4,"ABRAMS, SAMUEL A",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,35.60,1424.0


#### a) T python library formula (ttest_1samp)

In [422]:
H0_police_most_hourly = 35
α_most_hourly = 0.05

In [439]:
t_formula_most_hourly, t_pvalue_most_hourly = ttest_1samp(df_dept_most_hourly["Hourly Rate"], H0_police_most_hourly, alternative="less")
print("T value: " + str(t_formula_most_hourly))

T value: -9.567447887848152


In [438]:
uni_t_menor_most_hourly = scipy.stats.t.ppf((0.05),len(df_dept_most_hourly)-1) # H1 < H0
print("left T critical value: " + str(uni_t_menor_most_hourly))

left T critical value: -1.6456728276882668


In [432]:
"""We reject the hypothesis because T value is on the left of the T critical value"""

'We reject the hypothesis because T value is on the left of the T critical value'

#### b) Pvalue python library formula (ttest_1samp)

In [434]:
t_pvalue_most_hourly

1.6689265282353859e-21

In [468]:
if t_pvalue_most_hourly < α_most_hourly:
    print("We reject the hypothesis because P value is less than alfa")
else:
    print("We accept the hypothesis because P value is greater than alfa")

We reject the hypothesis because P value is less than alfa


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

## Exercise 1

In [299]:
df1_hourly.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Hourly Rate,Hourly Salary
0,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,19.86,397.2
1,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,46.10,1844.0
2,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,35.60,1424.0
3,"ABBOTT, BETTY L",FOSTER GRANDPARENT,FAMILY & SUPPORT,P,Hourly,20.0,2.65,53.0
4,"ABDULLAH, LAKENYA N",CROSSING GUARD,OEMC,P,Hourly,20.0,17.68,353.6


#### IC ZConfint

In [301]:
# IC Z INTERVAL with ZConfint:
IC3_hourly_z = zconfint(x1=df1_hourly["Hourly Rate"], alpha=0.05)
IC3_hourly_z

(32.52349834813791, 33.05361708442256)

In [303]:
"""in the first hypothesis of challenge 2 H0 was 30, so we should also reject the hypothesis because it is out of range"""

'in the first hypothesis of challenge 2 H0 was 30, so we should also reject the hypothesis because it is out of range'

##### IC T INTERVAL with t.interval:

In [457]:
t_interval_hourly = t.interval(df=len(df1_hourly["Hourly Rate"])-1, alpha=0.95, loc=df1_hourly["Hourly Rate"].mean(), scale=scipy.stats.sem(df1_hourly["Hourly Rate"]))

In [458]:
t_interval_hourly

(32.52345834488529, 33.05365708767727)

## Exercise 2

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [478]:
df_salaries_police_dept.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Annual Salary
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,101442.0
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,94122.0
2,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,93354.0
3,"ABDALLAH, ZAID",POLICE OFFICER,POLICE,F,Salary,84054.0
4,"ABDELHADI, ABDALMAHD",POLICE OFFICER,POLICE,F,Salary,87006.0


##### IC T INTERVAL with t.interval:

In [479]:
t_interval_salary = t.interval(df=len(df_salaries_police_dept["Annual Salary"])-1, alpha=0.95, loc=df_salaries_police_dept["Annual Salary"].mean(), scale=scipy.stats.sem(df_salaries_police_dept["Annual Salary"]))

In [480]:
t_interval_salary

(86177.05631531784, 86795.77269094894)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [476]:
# bilateral:
count = len(df1_hourly) #la parte del dataset a estudiar
nobs = len(df1) #todo el dataset
H0_bonus =  0.25
α_prop = 0.05

proportions_z, proportions_pvalue  = proportions_ztest(count, nobs, value=H0_bonus, alternative='two-sided')
proportions_pvalue

0.0004481127249057967

In [477]:
if proportions_pvalue < α_prop:
    print("We reject the hypothesis because P value is less than alfa")
else:
    print("We accept the hypothesis because P value is greater than alfa")

We reject the hypothesis because P value is less than alfa
